In [1]:
import requests
import string
import random
import time
import csv
import pandas as pd
import concurrent.futures
from bs4 import BeautifulSoup

In [3]:
# Function to fetch URLs for a character `alpha` from `base_url`
def fetch_urls(alpha):
    n_rows = 0
    urls = [[]]
    
    while(True):
        url = base_url + f'?letter={alpha}' + f'&show={n_rows}'
        time.sleep(random.uniform(0, 0.2))
        page = requests.get(url, headers=headers)

        # Fetch URLs from the page if there's no error
        if (page.status_code == 200):
            soup = BeautifulSoup(page.content, 'html.parser')
            for link in soup.find_all('a', class_='hoverinfo_trigger fw-b fl-l'):
                urls.append([link.get('href')])
            n_rows += 50

        if (page.status_code == 404):
            break

        if (page.status_code == 403):
            time.sleep(120)
    
    with open('./Datasets/anime-urls.csv', mode='a') as f:
        url_writer = csv.writer(f, delimiter=',', quotechar="'", quoting=csv.QUOTE_MINIMAL)
        url_writer.writerows(urls)

def fetch_all_urls(chars_list_=None):
    if chars_list_==None:
        chars_list_ = chars_list
    
    threads = min(MAX_THREADS, len(chars_list_))

    with concurrent.futures.ThreadPoolExecutor( max_workers=threads ) as executor:
        executor.map(fetch_urls, chars_list_)

In [4]:
# Init base URL & headers for get()
base_url = 'https://myanimelist.net/anime.php'
headers = {
    'Host': 'myanimelist.net',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:94.0) Gecko/20100101 Firefox/94.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://myanimelist.net/anime.php',
    'Connection': 'keep-alive'
}

# Init max number of threads
MAX_THREADS = 30

# Init list of characters
chars_list = ['.'] + list(string.ascii_uppercase)

# Create an empty CSV to store URLs
pd.DataFrame(list()).to_csv('./Datasets/anime-urls.csv')

fetch_all_urls(chars_list)